# ダウンロードしたデータセットに機械学習を適用する流れ

## 達成目標
- 学習器に対して
    - 機械学習のタスク種別がclassification/regression/clusteringのどれに該当するかを判断することができる。
    - 判断したタスクに用いることができる学習器（モデル）を選択することができる。
        - 参考
            - [Choosing the right estimator](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
            - [User Guide](https://scikit-learn.org/stable/user_guide.html)
            - [Examplex](https://scikit-learn.org/stable/auto_examples/index.html)
    - 選択したモデルのAPIを参照し、手動調整が必要なハイパーパラメータを確認することができる。
    - 選択したモデルのAPIを参照し、評価方法(score関数)を確認することができる。
- データセットに対して
    - ダウンロードしたデータセットの中身を目視確認し、保存されている形式を理解することができる。
    - 代表的な保存形式で保存されているデータセットに対して、pd.DataFrame形式として読み込むことができる。
        - 代表的な保存形式: csv, tsv
    - DataFrameから必要な行や列を指定してデータを抜き出すことができる。
        - 特徴ベクトルと教師データを分けたり、データセット全体を学習用・テスト用に分けることができる。
- 機械学習に対して
    - 学習データを学習器に与えて学習させ、テストデータで学習結果の適切さを評価することができる。
    - 学習データとテストデータに分割することができる。[交差検定(cross-validation)](https://scikit-learn.org/stable/modules/cross_validation.html)するとベター。


## 今回の流れ
- Machine Learning Repositoryで公開されている代表的なデータ[Iris Data Set](https://archive.ics.uci.edu/ml/datasets/Iris)に対し、[LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)で分類学習してみる。5分割検定で評価する。

### データセットの準備
データセットのページには iris.data, iris.names が用意されていることを確認。この2つのファイルをダウンロードして、中身を覗いてみよう。

In [1]:
# iris.data のダウンロード
!curl -O https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

# head コマンドにより、冒頭数行を確認する
!head iris.data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4551  100  4551    0     0   7157      0 --:--:-- --:--:-- --:--:--  7212
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa
4.9,3.1,1.5,0.1,Iris-setosa


- どうやら iris.data には「各値がカンマで区切られて列挙されたCSV形式」でデータセットが保存されているらしい。冒頭4つが特徴量で、最後の1つはラベルデータっぽい。CSVなので恐らく [pd.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) を使えば DataFrame 形式で読み込めるだろう。
- 次に iris.names を確認してみよう

In [2]:
!curl -O https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.names
!head iris.names

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2998  100  2998    0     0   4690      0 --:--:-- --:--:-- --:--:--  4721
1. Title: Iris Plants Database
	Updated Sept 21 by C.Blake - Added discrepency information

2. Sources:
     (a) Creator: R.A. Fisher
     (b) Donor: Michael Marshall (MARSHALL%PLU@io.arc.nasa.gov)
     (c) Date: July, 1988

3. Past Usage:
   - Publications: too many to mention!!!  Here are a few.


- どうやら iris.names はこのデータセットに関する説明が書かれているらしい。今回は使わない（説明ページで確認済み）ので、無視することにしよう。
- iris.data だけで揃うようなので、これを読み込んで処理する準備をしよう。

In [3]:
import pandas as pd
df = pd.read_csv('iris.data')
df

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


- pd.read_csvをそのまま使うと、1行目のデータを見出しとして処理してしまっている。これを避けるため、見出しなしとして再読み込みし直すか、もしくは別途見出しを用意して再読込し直す必要がありそうだ。ここでは見出しを用意することにしよう。

In [4]:
feature_names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class']
df = pd.read_csv('iris.data', names=feature_names)
df

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


- 学習に向けて特徴ベクトルと教師データを分けておく。

In [5]:
X = df[['sepal length', 'sepal width', 'petal length', 'petal width']]
y = df['class']
print('X.shape = ', X.shape)
print('y.shape = ', y.shape)
print(X.loc[0])
print(y[0])

X.shape =  (150, 4)
y.shape =  (150,)
sepal length    5.1
sepal width     3.5
petal length    1.4
petal width     0.2
Name: 0, dtype: float64
Iris-setosa


### 学習器の用意
- [LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)を参考にimportしておく。
- ハイパーパラメータのうち C について 0.5, 1.0, 1,5 の3通りを試してみよう。

In [6]:
from sklearn.svm import LinearSVC
Cs = [0.5, 1.0, 1.5]

### 交差検定を利用して学習＆評価
今回は[cross_val_score](https://scikit-learn.org/stable/modules/cross_validation.html#computing-cross-validated-metrics)を使ってみよう。

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

k_folds = 5 # 5分割検定

for c in Cs:
    model = LinearSVC(C=c)
    scores = cross_val_score(model, X, y, cv=KFold(n_splits=k_folds, shuffle=True))
    average = scores.mean()
    print(f'C = {c}: scores={scores}, average={average:.3f}')

C = 0.5: scores=[0.96666667 0.96666667 0.9        0.96666667 1.        ], average=0.960
C = 1.0: scores=[0.93333333 0.96666667 0.93333333 0.93333333 1.        ], average=0.953
C = 1.5: scores=[0.9        1.         0.93333333 1.         0.9       ], average=0.947


/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/tnal/.venv/dm/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear fail